## ME/EECS/BioE C106B/206B Homework 2



### Implementing Control Lyapunov Functions
In this notebook, you'll find the Python code necessary to compute the question "Implementing Control Lyapunov Functions." If you're having trouble installing libraries on your computer or if you'd rather run things on the cloud, feel free to use this notebook instead of the online version.\
To get started, go to "file" and then "save a copy in Drive." This will give you a copy of this notebook that you can edit.\
Each different file has been broken up into code cells in this notebook.
Have fun! :)

In [ ]:
#Install Dependencies
!pip install casadi

#set up matplotlib animation needed for notebook
from matplotlib import rc
rc('animation', html='jshtml')

# Navigating the Google Colab Environment
When you hover your mouse over a cell, a play button will appear on the left-hand side. By clicking the play button, you'll run the code in that cell. Every time you make a change to a code cell, you must run it to update it in the Google Colab environment!

The files you'll be editing in this problem are lyapunov.py and run_simulations.py, which you may find at the bottom of this page. You can easily navigate to these files by clicking the menu on the left hand side of the screen and selecting "Table of Contents." To test your code, run the run_simulation.py code cell. If you'd like to see an animated version of your result, run the animation cell below that one - note that this is optional and takes around a minute to run in the Colab environment.

### controller.py

In [ ]:
import numpy as np
import casadi as ca

"""
File containing controllers
"""
class Controller:
    def __init__(self, observer, lyapunov = None, trajectory = None, obstacleQueue = None, uBounds = None):
        """
        Skeleton class for feedback controllers
        Args:
            observer (Observer): state observer object
            lyapunov (LyapunovBarrier): lyapunov functions, LyapunovBarrier object
            trajectory (Trajectory): trajectory for the controller to track (could just be a constant point!)
            obstacleQueue (ObstacleQueue): ObstacleQueue object, stores all barriers for the system to avoid
            uBounds ((Dynamics.inputDimn x 2) numpy array): minimum and maximum input values to the system
        """
        #store input parameters
        self.observer = observer
        self.lyapunov = lyapunov
        self.trajectory = trajectory
        self.obstacleQueue = obstacleQueue
        self.uBounds = uBounds

        #store input
        self._u = None

    def eval_input(self, t):
        """
        Solve for and return control input
        Inputs:
            t (float): time in simulation
        Returns:
            u ((Dynamics.inputDimn x 1)): input vector, as determined by controller
        """
        self._u = np.zeros((self.observer.inputDimn, 1))
        return self._u

    def get_input(self):
        """
        Retrieves input stored in class parameter
        Returns:
            self._u: most recent input stored in class paramter
        """
        return self._u

class PlanarQrotorPD(Controller):
    def __init__(self, observer, lyapunov = None, trajectory = None, obstacleQueue = None, uBounds = None):
        """
        Init function for a planar quadrotor controller.

        Args:
            observer (Observer): state observer object
            lyapunov (LyapunovBarrier): lyapunov functions, LyapunovBarrier object
            trajectory (Trajectory): trajectory for the controller to track (could just be a constant point!)
            obstacleQueue (ObstacleQueue): ObstacleQueue object, stores all barriers for the system to avoid
            uBounds ((Dynamics.inputDimn x 2) numpy array): minimum and maximum input values to the system
        """
        super().__init__(observer, lyapunov = lyapunov, trajectory = trajectory, obstacleQueue = obstacleQueue, uBounds = uBounds)

        #Initialize variables for the gain parameters
        self.Kp = np.eye(3) #proportional position gain
        self.Kd = np.eye(3) #derivative position gain
        self.Ktheta = 1 #proportional orientation gain
        self.Komega = 1 #derivative orientation gain

        #Store quadrotor parameters from the observer
        self.m = self.observer.dynamics._m
        self.Ixx = self.observer.dynamics._Ixx
        self.g = 9.81 #store gravitational constant

        #store Euclidean basis vector
        self.e1 = np.array([[1, 0, 0]]).T
        self.e2 = np.array([[0, 1, 0]]).T
        self.e3 = np.array([[0, 0, 1]]).T

    def get_position_error(self, t):
        """
        Function to return the position error vector x_d - x_q
        Args:
            t (float): current time in simulation
        Returns:
            eX ((3 x 1) NumPy array): x_d - x_q based on current quadrotor state
        """
        #retrieve desired and current positions
        xD = self.trajectory.pos(t)
        xQ = self.observer.get_pos()

        #return difference
        return xD - xQ

    def get_velocity_error(self, t):
        """
        Function to return velocity error vector v_d - v_q
        Args:
            t (float): current time in simulation
        Returns:
            eX ((3 x 1) NumPY array): vD - vQ
        """
        #retrieve desired and current velocities
        vD = self.trajectory.vel(t)
        vQ = self.observer.get_vel()

        #return difference
        return vD - vQ

    def eval_force_vec(self, t):
        """
        Function to evaluate the force vector input to the system using point mass dynamics.
        Args:
            t (float): current time in simulation
        Returns:
            f ((3 x 1) NumPy Array): virtual force vector to be tracked by the orientation controller
        """
        #find position and velocity error
        eX = self.get_position_error(t)
        eV = self.get_velocity_error(t)

        #get desired acceleration from trajectory
        aD = self.trajectory.accel(t)

        #DEFINE YOUR P AND D GAINS
        self.Kp = np.eye(3)*4
        self.Kd = np.eye(3)*3

        #calculate control input - add feedforward acceleration term
        return self.Kp@eX + self.Kd@eV + self.m*self.g*self.e3 + self.m*aD

    def eval_desired_orient(self, f):
        """
        Function to evaluate the desired orientation of the system.
        Args:
            f ((3 x 1) NumPy array): force vector to track from point mass dynamics
        Returns:
            thetaD (float): desired angle of quadrotor WRT world frame
        """
        return np.arctan2(-f[1, 0], f[2, 0]) #remember to flip the sign!

    def eval_orient_error(self, t):
        """
        Evalute the orientation error of the system thetaD - thetaQ
        Args:
            t (float): current time in simulation
        Returns:
            eOmega (float): error in orientation angle
        """
        f = self.eval_force_vec(t) #force we'd like to track
        thetaD = self.eval_desired_orient(f) #desired angle of quadrotor
        thetaQ = self.observer.get_orient() #current angle of quadrotor

        #return the difference
        return thetaD - thetaQ

    def eval_moment(self, t):
        """
        Function to evaluate the moment input to the system
        Args:
            t (float): current time in simulation
        Returns:
            M (float): moment input to quadrotor
        """
        eTheta = self.eval_orient_error(t)
        eOmega = 0 - self.observer.get_omega() #assume zero angular velocity desired
        thetaDDotD = 0 #Assume a desired theta dddot of 0

        #DEFINE YOUR THETA AND OMEGA GAINS
        self.Ktheta = 0.08
        self.Komega = 0.02

        #return the PD controller output - assume zero desired angular acceleration
        return self.Ktheta*eTheta + self.Komega*eOmega + self.Ixx*thetaDDotD

    def eval_force_scalar(self, t):
        """
        Evaluates the scalar force input to the system.
        Args:
            t (float): current time in simulation
        Returns:
            F (float): scalar force input from PD control
        """
        #first, construct R, a rotation matrix about the x axis
        thetaQ = self.observer.get_orient()
        R = np.array([[1, 0, 0],
                      [0, np.cos(thetaQ), -np.sin(thetaQ)],
                      [0, np.sin(thetaQ), np.cos(thetaQ)]])

        #find and return the scalar force
        return (self.eval_force_vec(t).T@R@self.e3)[0, 0]

    def eval_input(self, t):
        """
        Get the control input F, M to the planar quadrotor system
        Args:
            t (float): current time in simulation
        Returns:
            self._u = [F, M] ((2x1) numpy array): force, moment input to system
        """
        #store input in the class parameter
        self._u = np.array([[self.eval_force_scalar(t), self.eval_moment(t)]]).T
        return self._u


class PlanarQrotorLyapunov(PlanarQrotorPD):
    """
    Class for a CLF-QP tracking controller, inherits from PlanarQrotorPD
    """
    def __init__(self, observer, lyapunov = None, trajectory = None, obstacleQueue = None, uBounds = None, clfGamma = 0.1):
        """
        Init function for a planar quadrotor CLF-QP tracking controller.
        Inherits from PD class, and uses PD orientation contorl, ES-CLF-QP tracking control
        Inputs:
            clfGamma (float): constant for ES-CLF rate of convergence
        """
        super().__init__(observer, lyapunov = lyapunov, trajectory = trajectory, obstacleQueue = obstacleQueue, uBounds = uBounds)
        self.clfGamma = clfGamma

    def eval_force_vec(self, t):
        """
        Evaluates the force vector input to the system at time t
        Args:
            t (float): current time in simulation
        Returns:
            f ((3 x 1) NumPy Array): virtual force vector to be tracked by the orientation controller
        """
        #set up optimization problem
        opti = ca.Opti()

        #set up optimization variables
        u = opti.variable(3, 1) #the input here is a force vector only! Will be 3D.

        #get the CLF value and its derivative
        V = self.lyapunov.evalLyapunov(t)
        VDot = self.lyapunov.evalLyapunovDerivs(u, t) #pass the input into the function

        #Apply CLF constraint to problem
        opti.subject_to(VDot <= -self.clfGamma*V)

        #Define Cost Function
        H = np.eye(3)
        cost = ca.mtimes(u.T, ca.mtimes(H, u))

        #set up optimization problem
        opti.minimize(cost)
        option = {"verbose": False, "ipopt.print_level": 0, "print_time": 0}
        opti.solver("ipopt", option)

        #solve optimization
        try:
            sol = opti.solve()
            uOpt = sol.value(u) #extract optimal input
            solverFailed = False
        except:
            print("Solver failed!")
            solverFailed = True
            uOpt = np.zeros((3, 1)) #return a zero vector

        #store output in class param - add on the effect of gravit here!
        self._u = uOpt.reshape((3, 1)) + self.m*self.g*self.e3

        #return result
        return self._u

### trajectory.py

In [ ]:
import time

class Trajectory:
    def __init__(self, start, end, T):
        """
        Init function for linear tracking trajectories in RN.
        Generates a smooth straight line trajectory with zero start and end velocity. Uses sinusoidal interpolation.
        Args:
            start (Nx1 numpy array): initial spatial position in N dimensions (NOT initial state vector)
            end (Nx1 numpy array): final spatial position in N dimensions
            T (float): trajectory period
        """
        self.x0 = start
        self.xF = end
        self.spatialDimn = self.x0.shape[0]
        self.T = T

    def pos(self, t):
        """
        Function to get desired position at time t
        Args:
            t (float): current time
        Returns:
            (Nx1 numpy array): position coordinates for the quadrotor to track at time t
        """
        #use sinusoidal interpolation to get a smooth trajectory with zero velocity at endpoints
        if t>self.T:
            #if beyond the time of the trajectory end, return the desired position as a setpoint
            return self.xF
        des_pos = (self.xF-self.x0)/2*np.sin(t*np.pi/self.T - np.pi/2)+(self.x0+self.xF)/2
        return des_pos #calculates all three at once

    def vel(self, t):
        """
        Function to get the desired velocity at time t
        Inputs:
            t: current time
        Returns:
            (Nx1 Numpy array): velocity for the system to track at time t
        """
        #differentiate position
        if t>self.T:
            #If beyond the time of the trajectory end, return 0 as desired velocity
            return np.zeros((self.spatialDimn, 1))
        des_vel = (self.xF-self.x0)/2*np.cos(t*np.pi/self.T - np.pi/2)*np.pi/self.T
        return des_vel

    def accel(self, t):
        """
        Function to get the desired acceleration at time t
        Args:
            t: current time
        Returns:
            (Nx1 Numpy array): acceleration for the system to track at time t
        """
        #differentiate acceleration
        if t>self.T:
            #If beyond the time of the trajectory end, return 0 as desired acceleration
            return np.zeros((self.spatialDimn, 1))
        des_accel = -(self.xF-self.x0)/2*np.sin(t*np.pi/self.T - np.pi/2)*(np.pi/self.T)**2
        return des_accel

    def get_state(self, t):
        """
        Function to get the desired position, velocity, and accel at a time t
        Inputs:
            t: current time
        Returns:
            x_d, v_d, a_d: desired position, velocity, and acceleration at time t
        """
        return self.pos(t), self.vel(t), self.accel(t)

### state_estimation.py

In [ ]:
import numpy as np
from scipy.spatial import cKDTree

class StateObserver:
    def __init__(self, dynamics, mean = None, sd = None):
        """
        Init function for state observer

        Args:
            dynamics (Dynamics): Dynamics object instance
            mean (float, optional): Mean for gaussian noise. Defaults to None.
            sd (float, optional): standard deviation for gaussian noise. Defaults to None.
        """
        self.dynamics = dynamics
        self.stateDimn = dynamics.stateDimn
        self.inputDimn = dynamics.inputDimn
        self.mean = mean
        self.sd = sd

    def get_state(self):
        """
        Returns a potentially noisy observation of the system state
        """
        if self.mean or self.sd:
            #return an observation of the vector with noise
            return self.dynamics.get_state() + np.random.normal(self.mean, self.sd, (self.stateDimn, 1))
        return self.dynamics.get_state()

class QuadObserver(StateObserver):
    def __init__(self, dynamics, mean, sd):
        """
        Init function for state observer for a planar quadrotor

        Args:
            dynamics (Dynamics): Dynamics object instance
            mean (float, optional): Mean for gaussian noise. Defaults to None.
            sd (float, optional): standard deviation for gaussian noise. Defaults to None.
        """
        super().__init__(dynamics, mean, sd)

    def get_pos(self):
        """
        Returns a potentially noisy measurement of JUST the position of the Qrotor mass center
        Returns:
            3x1 numpy array, observed position vector of system
        """
        return self.get_state()[0:3].reshape((3, 1))

    def get_vel(self):
        """
        Returns a potentially noisy measurement of JUST the spatial velocity of the Qrotor mass center
        Returns:
            3x1 numpy array, observed velocity vector of system
        """
        return self.get_state()[4:7].reshape((3, 1))

    def get_orient(self):
        """
        Returns a potentially noisy measurement of the
        Assumes that the system is planar and just rotates about the X axis.
        Returns:
            theta (float): orientation angle of quadrotor with respect to world frame
        """
        return self.get_state()[3, 0]

    def get_omega(self):
        """
        Returns a potentially noisy measurement of the angular velocity theta dot
        Assumes that the system is planar and just rotates about the X axis.
        Returns:
            theta (float): orientation angle of quadrotor with respect to world frame
        """
        return self.get_state()[7, 0]

### dynamics.py

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation

class Dynamics:
    """
    Skeleton class for system dynamics
    Includes methods for returning state derivatives, plots, and animations
    """
    def __init__(self, x0, stateDimn, inputDimn, relDegree = 1):
        """
        Initialize a dynamics object
        Args:
            x0 (stateDimn x 1 numpy array): initial condition state vector
            stateDimn (int): dimension of state vector
            inputDimn (int): dimension of input vector
            relDegree (int, optional): relative degree of system. Defaults to 1.
        """
        self.stateDimn = stateDimn
        self.inputDimn = inputDimn
        self.relDegree = relDegree

        #store the state and input
        self._x = x0
        self._u = None

    def get_state(self):
        """
        Retrieve the state vector
        """
        return self._x

    def deriv(self, x, u, t):
        """
        Returns the derivative of the state vector
        Args:
            x (stateDimn x 1 numpy array): current state vector at time t
            u (inputDimn x 1 numpy array): current input vector at time t
            t (float): current time with respect to simulation start
        Returns:
            xDot: state_dimn x 1 derivative of the state vector
        """
        return np.zeros((self.state_dimn, 1))

    def integrate(self, u, t, dt):
        """
        Integrates system dynamics using Euler integration
        Args:
            u (inputDimn x 1 numpy array): current input vector at time t
            t (float): current time with respect to simulation start
            dt (float): time step for integration
        Returns:
            x (stateDimn x 1 numpy array): state vector after integrating
        """
        #integrate starting at x
        self._x = self.get_state() + self.deriv(self.get_state(), u, t)*dt
        return self._x

    def get_plots(self, x, u, t):
        """
        Function to show plots specific to this dynamic system.
        Args:
            x ((stateDimn x N) numpy array): history of N states to plot
            u ((inputDimn x N) numpy array): history of N inputs to plot
            t ((1 x N) numpy array): history of N times associated with x and u
        """
        pass

    def show_animation(self, x, u, t):
        """
        Function to play animations specific to this dynamic system.
        Args:
            x ((stateDimn x N) numpy array): history of N states to plot
            u ((inputDimn x N) numpy array): history of N inputs to plot
            t ((1 x N) numpy array): history of N times associated with x and u
        """
        pass

"""
**********************************
PLACE YOUR DYNAMICS FUNCTIONS HERE
**********************************
"""
class QuadDyn(Dynamics):
    def __init__(self, x0 = np.zeros((8, 1)), stateDimn = 8, inputDimn = 2, relDegree = 2, m = 0.92, Ixx = 0.0023, l = 0.15):
        """
        Init function for a Planar quadrotor system.
        State Vector: X = [x, y, z, theta, x_dot, y_dot, z_dot, theta_dot]
        Input Vector: U = [F, M]

        Args:
            x0 ((8 x 1) NumPy Array): initial state (x, y, z, theta, x_dot, y_dot, z_dot, theta_dot)
            stateDimn (int): dimension of state vector
            inputDimn (int): dimension of input vector
            relDegree (int, optional): relative degree of system
            m (float): mass of quadrotor in kg
            Ixx (float): moment of inertia about x axis of quadrotor
            l (float): length of one arm of quadrotor
        """
        super().__init__(x0, stateDimn, inputDimn, relDegree)

        #store physical parameters
        self._m = m
        self._Ixx = Ixx
        self._g = 9.81
        self._l = l

        #store an animation reference for Colab
        self.anim = None

    def deriv(self, X, U, t):
        """
        Returns the derivative of the state vector
        Args:
            X (8 x 1 numpy array): current state vector at time t
            U (2 x 1 numpy array): current input vector at time t
            t (float): current time with respect to simulation start
        Returns:
            xDot: state_dimn x 1 derivative of the state vector
        """
        #unpack the input vector
        F, M = U[0, 0], U[1, 0]
        F = max(0, F) #CUT OFF THE FORCE AT ZERO!

        #unpack the state vector
        x_dot, y_dot, z_dot = X[4, 0], X[5, 0], X[6, 0] #velocities
        theta, theta_dot = X[3, 0], X[7, 0] #orientations

        #calculate the second time derivatives of each
        x_ddot = 0
        y_ddot = (-np.sin(theta)*F)/self._m
        z_ddot = (np.cos(theta)*F - self._m*self._g)/self._m
        theta_ddot = M/self._Ixx

        #construct and return state vector
        return np.array([[x_dot, y_dot, z_dot, theta_dot, x_ddot, y_ddot, z_ddot, theta_ddot]]).T

    def show_animation(self, xData, uData, tData, animate = True):
        """
        Shows the animation and visualization of data for this system.
        Args:
            xData (stateDimn x N Numpy array): state vector history array
            u (inputDimn x N numpy array): input vector history array
            t (1 x N numpy array): time history
            animate (bool, optional): Whether to generate animation or not. Defaults to True.
        """
        #Set constant animtion parameters
        GOAL_POS = [1, 2]
        FREQ = 50 #control frequency, same as data update frequency
        L = 0.14 #quadrotor arm length

        if animate:
            fig, ax = plt.subplots()
            # set the axes limits
            ax.axis([-1, 2.5, 0, 2.5])
            # set equal aspect such that the circle is not shown as ellipse
            ax.set_aspect("equal")
            # create a point in the axes
            point, = ax.plot(0,1, marker="o")
            num_frames = xData.shape[1]-1

            #define the line for the quadrotor
            line, = ax.plot([], [], 'o-', lw=2)

            #plot the goal position
            ax.scatter([GOAL_POS[0]], [GOAL_POS[1]], color = 'y')

            def animate(i):
                y = xData[1, i]
                z = xData[2, i]
                point.set_data(y, z)

                #draw the quadrotor line body
                theta = xData[3, i]
                x1 = y + L*np.cos(theta)
                x2 = y - L*np.cos(theta)
                y1 = z + L*np.sin(theta)
                y2 = z - L*np.sin(theta)
                thisx = [x1, x2]
                thisy = [y1, y2]
                line.set_data(thisx, thisy)

                return line, point

            anim = animation.FuncAnimation(fig, animate, frames=num_frames, interval=1/FREQ*1000, blit=True)
            self.anim = anim
            plt.xlabel("Y Position (m)")
            plt.ylabel("Z Position (m)")
            plt.title("Position of Drone in Space")
            plt.show()

        #Plot each state variable in time
        fig, axs = plt.subplots(6)
        fig.suptitle('Evolution of States in Time')
        xlabel = 'Time (s)'
        ylabels = ['Y Pos (m)', 'Z Pos (m)', 'Theta (rad)', 'Y Vel (m/s)', 'Z Vel (m/s)', 'Angular Vel (rad/s)']
        indices = [1, 2, 3, 5, 6, 7] #skip the x coordinates - indices in the state vector to plot
        goalStates = [0, 1, 2, 0, 0, 0, 0, 0]
        n = 0 #index in the subplot
        #plot the states
        for i in indices:
            axs[n].plot(tData.reshape((tData.shape[1], )).tolist(), xData[i, :].tolist())
            #plot the goal state for each
            axs[n].plot(tData.reshape((tData.shape[1], )).tolist(), [goalStates[i]]*tData.shape[1], 'r:')
            axs[n].set(ylabel=ylabels[n]) #pull labels from the list above
            axs[n].grid()
            n += 1
        axs[5].set(xlabel = xlabel)
        plt.show()
        #plot the inputs in a new plot
        fig, axs = plt.subplots(2)
        fig.suptitle('Evolution of Inputs in Time')
        xlabel = 'Time (s)'
        ylabels = ['Force (N)', 'Moment (N*m)']
        for i in range(2):
            axs[i].plot(tData.reshape((tData.shape[1], )).tolist(), uData[i, :].tolist())
            axs[i].set(ylabel=ylabels[i])
            axs[i].grid()
        axs[1].set(xlabel = xlabel)
        plt.show()

### environment.py

In [ ]:
class Environment:
    def __init__(self, dynamics, controller, observer):
        """
        Initializes a simulation environment
        Args:
            dynamics (Dynamics): system dynamics object
            controller (Controller): system controller object
            observer (Observer): system state estimation object
        """
        #store system parameters
        self.dynamics = dynamics
        self.controller = controller
        self.observer = observer

        #define environment parameters
        self.iter = 0 #number of iterations
        self.t = 0 #time in seconds
        self.done = False

        #Store system state
        self.x = self.dynamics.get_state() #Actual state of the system
        self.x0 = self.x #store initial condition for use in reset
        self.xObsv = None #state as read by the observer

        #Define simulation parameters
        self.SIM_FREQ = 1000 #integration frequency in Hz
        self.CONTROL_FREQ = 50 #control frequency in Hz
        self.SIMS_PER_STEP = self.SIM_FREQ//self.CONTROL_FREQ
        self.TOTAL_SIM_TIME = 10 #total simulation time in s

        #Define history arrays
        self.xHist = np.zeros((self.dynamics.stateDimn, self.TOTAL_SIM_TIME*self.CONTROL_FREQ))
        self.uHist = np.zeros((self.dynamics.inputDimn, self.TOTAL_SIM_TIME*self.CONTROL_FREQ))
        self.tHist = np.zeros((1, self.TOTAL_SIM_TIME*self.CONTROL_FREQ))

    def reset(self):
        """
        Reset the gym environment to its inital state.
        """
        #Reset gym environment parameters
        self.iter = 0 #number of iterations
        self.t = 0 #time in seconds
        self.done = False

        #Reset system state
        self.x = self.x0 #retrieves initial condiiton
        self.xObsv = None #reset observer state

        #Define history arrays
        self.xHist = np.zeros((self.dynamics.stateDimn, self.TOTAL_SIM_TIME*self.CONTROL_FREQ + 1))
        self.uHist = np.zeros((self.dynamics.inputDimn, self.TOTAL_SIM_TIME*self.CONTROL_FREQ + 1))
        self.tHist = np.zeros((1, self.TOTAL_SIM_TIME*self.CONTROL_FREQ + 1))

    def step(self):
        """
        Step the sim environment by one integration
        """
        #retrieve current state information
        self._get_observation() #updates the observer

        #solve for the control input using the observed state
        self.controller.eval_input(self.t)

        #Zero order hold over the controller frequency
        for i in range(self.SIMS_PER_STEP):
            self.dynamics.integrate(self.controller.get_input(), self.t, 1/self.SIM_FREQ) #integrate dynamics
            self.t += 1/self.SIM_FREQ #increment the time

        #update the deterministic system data, iterations, and history array
        self._update_data()

    def _update_data(self):
        """
        Update history arrays and deterministic state data
        """
        #append the input, time, and state to their history queues
        self.xHist[:, self.iter] = self.x.reshape((self.dynamics.stateDimn, ))
        self.uHist[:, self.iter] = (self.controller.get_input()).reshape((self.dynamics.inputDimn, ))
        self.tHist[:, self.iter] = self.t

        #update the actual state of the system
        self.x = self.dynamics.get_state()

        #update the number of iterations of the step function
        self.iter +=1

    def _get_observation(self):
        """
        Updates self.xObsv using the observer data
        Useful for debugging state information.
        """
        self.xObsv = self.observer.get_state()
        # print("current orientation: ", self.observer.get_orient())

    def _get_reward(self):
        """
        Calculate the total reward for ths system and update the reward parameter.
        Only implement for use in reinforcement learning.
        """
        return 0

    def _is_done(self):
        """
        Check if the simulation is complete
        Returns:
            boolean: whether or not the time has exceeded the total simulation time
        """
        #check current time with respect to simulation time
        if self.t >= self.TOTAL_SIM_TIME:
            return True
        return False

    def run(self, N = 1):
        """
        Function to run the simulation N times
        Inputs:
            N (int): number of simulation examples to run
        """
        #loop over an overall simulation N times
        for i in range(N):
            self.reset()
            while not self._is_done():
                print("Simulation Time Remaining: ", self.TOTAL_SIM_TIME - self.t)
                self.step() #step the environment while not done
            self.visualize() #render the result

    def visualize(self):
        """
        Provide visualization of the environment
        """
        self.dynamics.show_animation(self.xHist, self.uHist, self.tHist)

### lyapunov.py

In [ ]:
class Lyapunov:
    """
    Skeleton class for Lyapunov functions.
    Includes utilities to get Lyapunov values and Lyapunov derivatives
    """
    def __init__(self, stateDimn, inputDimn, observer, relDegree):
        """
        Init function for a Lyapunov/Barrier object
        Args:
            stateDimn (int): length of state vector
            inputDimn (int): length of input vector
            observer (observer): observer object
            relDegree (int): relative degree of Lyapunov function
        """
        self.stateDimn = stateDimn
        self.inputDimn = inputDimn
        self.observer = observer
        self.relDegree = relDegree

        #parameters to store values
        self._vals = None #stored derivatives + value of function (initialize as None)

    def evalLyapunov(self, t):
        """
        Evaluates the Lyapunov function at time t.
        Args:
            t (float): current time in simulation
        Returns:
            V(x, t): value of Lyapunov function at time t, state x
        """
        return 0

    def evalLyapunovDerivs(self, u, t):
        """
        Returns the r derivatives of the Lyapunov function
        Args:
            Args:
            u (numpy array, (input_dimn x 1)): current input vector to system
            t (float): current time in simulation
        Returns:
        [..., vDDot, vDot, v] ((self.dynamics.relDegree + 1, 1) numpy Array): dynamics.relDegree lyapunov/Barrier time derivs, Descending order
        """
        return np.zeros((self.observer.dynamics.relDegree + 1, 1))


    def eval(self, u, t):
        """
        Returns a list of derivatives (going until zeroth derivative) of the lyapunov/Barrier function.
        Args:
            u (numpy array, (input_dimn x 1)): current input vector to system
            t (float): current time in simulation
        Returns:
        [..., vDDot, vDot, v] ((self.dynamics.relDegree + 1, 1) numpy Array): dynamics.relDegree lyapunov/Barrier time derivs, Descending order
        """
        self._vals = np.vstack((self.evalLyapunov(t), self.evalLyapunovDerivs(u, t)))
        return self._vals

    def get(self):
        """
        Retreives stored function and derivative values
        Returns:
        [..., vDDot, vDot, v] ((self.observer.dynamics.relDegree + 1, 1) numpy Array): dynamics.relDegree lyapunov/Barrier time derivs, Descending order
        """
        return self._vals

"""
********************************
ADD YOUR LYAPUNOV FUNCTIONS HERE
********************************
"""
class LyapunovQrotor(Lyapunov):
    """
    Lyapunov function for point mass trajectory tracking in the quadrotor
    """
    def __init__(self, stateDimn, inputDimn, observer, traj):
        """
        Init function for a Lyapunov/Barrier object
        Args:
            stateDimn (int): length of state vector
            inputDimn (int): length of input vector
            observer (QuadObserver): QuadObserver object
            traj (Trajectory): Trajectory object (required for tracking)
        """
        #initialize skeleton object. The relative degree of this system is 1.
        super().__init__(stateDimn, inputDimn, observer, 1)

        #store the trajectory in a class parameter
        self.traj = traj

        #get the mass of the system from the observer
        self.m = self.observer.dynamics._m

        #store the Lyapunov tuning constants (selected to keep Vx quadratic)
        a = 1 #a may be tuned
        b = (self.m/2)**0.5 #b must be a fixed constant
        self.alpha = 2*a**2
        self.epsilon = 2*(a+b)

    def evalLyapunov(self, t):
        """
        Evaluates the Lyapunov function at time t.
        Args:
            t (float): current time in simulation
        Returns:
            V(x, t) (float): value of Lyapunov function at time t, state x
        """
        #get the position and velocity from the observer
        x = self.observer.get_pos()
        v = self.observer.get_vel()

        #get desired position, velocity, and accel from the trajectory
        xD, vD, aD = self.traj.get_state(t)

        #define position and velocity errors
        eX = x - xD
        eV = v - vD

        #extract alpha, epsilon, and mass
        alpha = self.alpha
        epsilon = self.epsilon
        m = self.m

        """
        ***************************************
        YOUR CODE HERE: Here, you should calculate and return the value of the Lyapunov function, V(x)
        Above, several useful parameters have been defined for you! Your code should return a single float value.
        ***************************************
        """

        return 0 #TODO: Change this to return the value of the Lyapunov function.

    def evalLyapunovDerivs(self, f, t):
        """
        Evaluates the first derivative of the Lyapunov function at time t.
        Args:
            f ((inputDimn x 1) numPy array): input to the system
            t (float): current time in simulation
        Returns:
            Vdot(x, t): first time derivative of Lyapunov function at time t, state x
        """
        #get the position and velocity from the observer
        x = self.observer.get_pos()
        v = self.observer.get_vel()

        #get desired position, velocity, and accel from the trajectory
        xD, vD, aD = self.traj.get_state(t)

        #define position and velocity errors
        eX = x - xD
        eV = v - vD

        #extract alpha, epsilon, and mass
        alpha = self.alpha
        epsilon = self.epsilon
        m = self.m

        """
        ***************************************
        YOUR CODE HERE: Here, you should calculate and return the value of the first time derivative of the Lyapunov function
        Above, several useful parameters have been defined for you!
        NOTE: In this function, Do not use np.matmul for multiplication, use the @ symbol. To find the square of the norm of a vector, use x.T @ x.
        ***************************************
        """

        return 0 #TODO: update this line

### run_simulation.py

In [ ]:
#system initial condition
x0 = np.array([[0, 0, 1, 0, 0, 0, 0, 0]]).T #start the quadrotor at 1 M in the air

#create a dynamics object for the double integrator
dynamics = QuadDyn(x0)

#create an observer based on the dynamics object with noise parameters
mean = 0
sd = 0.01
observer = QuadObserver(dynamics, mean, sd)

#create a trajectory
start = np.array([[0, 0, 1]]).T #Pass in simply spatial dimensions into the system
end = np.array([[0, 1, 2]]).T #goal state in space
T = 3 #Period of trajectory
trajectory = Trajectory(start, end, T)

#create a lyapunov function for a 3D particle with a force vector input
lyap = LyapunovQrotor(6, 3, observer, trajectory)

#define CLF gamma
gamma = 0 #TODO: YOU SHOULD TUNE THIS VALUE HERE TO ENSURE SMOOTH TRAJECTORY TRACKING

#create a planar quadrotor controller
controller = PlanarQrotorLyapunov(observer, lyapunov = lyap, trajectory = trajectory, clfGamma = gamma)

#create a simulation environment
env = Environment(dynamics, controller, observer)
env.reset()

#run the simulation
env.run()

## Run Animation (Optional)
Run the code cell below to visualize the car's motion. Note that this simply animates the path that's generated in the graph above, and is in a separate cell here due to a quirk of Google Colab. Note that it might take a little while to run!

In [ ]:
dynamics.anim